In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, CovarianceStructure, ObjectiveLib
#import matplotlib.pyplot as plt
import pickle 

In [2]:
Nt = 1

# they are set up but not work
max_manual_num = 100
min_interval_num = 100

In [3]:
#measure_names = ["y_CO2", "C_CO2", "C_N2", "Tg", "Pg", "vg", "q_CO2","Ts", "y_theta"]
measure_names = ["Tg", "Pg", "vg", "Ts", "y_CO2",  "C_CO2", "C_N2",  "q_CO2","y_theta"]
num_measure = len(measure_names)

#z_pos = 8
#z_names = "01234567"
z_pos = 3
z_names = "012"

o_pos = 3 
o_names = "012"

num_mod = 2 
mod_names = "ads", "des"

num_total = num_measure*z_pos*o_pos*num_mod

all_ind = list(range(num_total))

In [4]:
# name generator
# CCO2, ADS, Z=0, O=0
# CCO2, ADS, Z=1, O=0
# ...
# CCO2, ADS, Z=0, O=1
#...
# CCO2, DES, Z=0, O=0
# CCO2, DES, Z=1, O=0
# ...
# CCO2, DES, Z=0, O=1
# adsorption + desorption, ...

def name_generator():
    
    name_list = []
    static_costs = []
    dynamic_costs = []
    
    
    for m_n in measure_names:
        for mod_n in mod_names:
            if m_n in ["Tg", "Pg", "vg", "Ts"]:
                static = 1000
                dynamic = 0 
            else:
                static = 0 
                dynamic = 100 
            
            for o in list(o_names):
                for z in list(z_names):
                    name = "_".join([m_n, z, o, mod_n])
                    
                    name_list.append(name)
                    static_costs.append(static)
                    dynamic_costs.append(dynamic)
    
    return name_list, static_costs, dynamic_costs

all_names, static_cost, dynamic_cost = name_generator() 
print(dynamic_cost)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


In [5]:
num_z_o = z_pos*o_pos     

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

In [6]:
measure_info = pd.DataFrame({
    "name": all_names,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

                name  Q_index  static_cost  dynamic_cost  min_time_interval  \
0         Tg_0_0_ads        0         1000             0                100   
1         Tg_1_0_ads        1         1000             0                100   
2         Tg_2_0_ads        2         1000             0                100   
3         Tg_0_1_ads        3         1000             0                100   
4         Tg_1_1_ads        4         1000             0                100   
..               ...      ...          ...           ...                ...   
157  y_theta_1_1_des      157            0           100                100   
158  y_theta_2_1_des      158            0           100                100   
159  y_theta_0_2_des      159            0           100                100   
160  y_theta_1_2_des      160            0           100                100   
161  y_theta_2_2_des      161            0           100                100   

     max_manual_number  
0                  100  
1

In [7]:
num_static = 4*num_mod*z_pos*o_pos
num_dynamic  = 5*num_mod*z_pos*o_pos

print(num_static, num_dynamic)
num_total = num_static+num_dynamic

72 90


In [9]:
dataObject = DataProcess()
dataObject.read_jacobian('./MO_QVs/Q_z3')
Q = dataObject.get_Q_list(list(range(num_static)), list(range(num_dynamic)), Nt)

print(np.shape(Q))

print(Q[1])

(162, 1, 5)
[[0.01424043941256059, -0.09380282162396725, -0.06041254277988628, 0.04913281242124867, -0.0004855924942148704]]


In [10]:
## error covariance 
file = open("./MO_QVs/Var_z3", "rb")
data = pickle.load(file)
file.close()

error_cov = np.zeros((num_total, num_total))

for i in range(num_total):
    error_cov[i,i] = data[i]

In [12]:
calculator = MeasurementOptimizer(Q, measure_info, error_cov=error_cov, error_opt=CovarianceStructure.measure_correlation, verbose=True)


fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

Covariances are between measurements at the same time.
(26244, 5, 5)


[[0.011414240197826785, -0.06912939982116605, -0.034850453503580174, 0.024804195011498376, 0.0003070317130399114], [-0.06912939982116605, 0.41867648102801497, 0.21106888346862096, -0.15022455147899497, -0.0018595121252621503], [-0.034850453503580174, 0.21106888346862096, 0.10640691700499247, -0.07573324461023205, -0.0009374425501803672], [0.024804195011498373, -0.15022455147899494, -0.07573324461023205, 0.05390179981367319, 0.0006672081849483382], [0.00030703171303991133, -0.0018595121252621503, -0.0009374425501803673, 0.0006672081849483382, 8.258847823280495e-06]]


In [14]:
#with open('./kinetics_results/May2_3000_a', 'rb') as f:
#    init_cov_y = pickle.load(f)
#print(init_cov_y)

fim_prior = [[0]*5 for _ in range(5)]

for _ in range(5):
    fim_prior[_][_] = 1 
    
fim_prior = np.asarray(fim_prior)

In [15]:
mip_option = False
objective = ObjectiveLib.A
sparse_opt = True
fix_opt = False

manual_num = 100
budget_opt = 10000

total_manual_init = 0
dynamic_install_init = [1]*num_total 

num_dynamic_time = [0]

#static_dynamic = [[0,3],[1,4],[2,5]]
#time_interval_for_all = True

#dynamic_time_dict = {}
#for i, tim in enumerate(num_dynamic_time[1:]):
#    dynamic_time_dict[i] = np.round(tim, decimals=2)
    
#print(dynamic_time_dict)

In [ ]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    upper_diagonal_only=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    #init_cov_y= init_cov_y,
                    initial_fim = fim_prior,
                    #dynamic_install_initial = dynamic_install_init, 
                    #static_dynamic_pair=static_dynamic,
                    #time_interval_all_dynamic = time_interval_for_all,
                    total_manual_num_init=total_manual_init)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

Ipopt 3.13.2: linear_solver=ma57


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        compu

  69 -6.7435163e+11 1.83e-04 1.45e+08  -1.0 2.98e+00    -  3.46e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70 -6.7435163e+11 6.10e-05 1.22e-04  -1.0 2.12e+00    -  1.00e+00 1.00e+00h  1
  71 -6.7435164e+11 1.83e-04 2.02e+02  -5.7 4.72e+02    -  3.65e-01 6.04e-01f  1
Reallocating memory for MA57: lfact (21161963)
  72 -6.7435164e+11 3.05e-04 4.64e+02  -5.7 4.42e+02    -  9.41e-02 3.28e-01f  1
  73 -6.7435164e+11 3.66e-04 3.95e+02  -5.7 4.56e+02    -  1.50e-01 1.51e-01f  1
  74 -6.7435164e+11 2.44e-04 1.75e+02  -5.7 3.39e+02    -  1.91e-01 8.33e-02f  1
Reallocating memory for MA57: lfact (23027194)
  75 -6.7435164e+11 2.00e-04 2.75e+02  -5.7 3.55e+02    -  1.51e-01 2.41e-01f  1
  76 -6.7435164e+11 4.88e-04 3.22e+02  -5.7 3.48e+02    -  2.01e-01 2.71e-01f  1
Reallocating memory for MA57: lfact (25457761)
  77 -6.7435164e+11 1.83e-04 5.93e+02  -5.7 2.78e+02    -  1.62e-01 3.79e-01f  1
  78 -6.7435164e+11 9.78e-05 2.45e+02  -5.7 1.55e

In [ ]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

ans_y, sol_y = calculator.extract_solutions(mod)
print('pyomo calculated cost:', pyo.value(mod.cost))

### verify 

In [ ]:
## rank FIMs 

measure_names = [
    'Tg_0', 'Tg_1', 'Tg_2', 'Pg_0', 'Pg_1', 'Pg_2', 'vg_0', 'vg_1', 'vg_2', 
    'Ts_0', 'Ts_1', 'Ts_2',
    'y_CO2_0', 'y_CO2_1', 'y_CO2_2', 'C_CO2_0', 'C_CO2_1', 
                 'C_CO2_2', 'C_N2_0', 'C_N2_1', 'C_N2_2', 
                 'qCO2_0', 'qCO2_1', 'qCO2_2',  'ytheta_0', 'ytheta_1', 'ytheta_2']

measure_lookup = {}

for i, n in enumerate(measure_names):
    measure_lookup[n] = i  


idx_lookup_solid = [1,5,10]
idx_lookup_gas = [5,20,40]

num_t = len(idx_lookup_solid)

In [ ]:
i=7

idx = i*num_total+i 

print(calculator.fim_collection[idx])

In [ ]:
### If it's computing right FIMs 

def verify_FIM(name, time_idx): 
    
    if name in ['qCO2_0', 'qCO2_1', 'qCO2_2', 'Ts_0', 'Ts_1', 'Ts_2', 'ytheta_0', 'ytheta_1', 'ytheta_2']:
        lookup = idx_lookup_solid 
        
    else:
        lookup = idx_lookup_gas
       
    # index of this name+time index in MO package
    idx_converted = measure_lookup[name]*num_t+lookup.index(time_idx)
    print("should be the ", idx_converted)
    
    # find computed FIMs 
    file = open("./MO_FIMs/"+name, "rb")
    data = pickle.load(file)
    file.close()

    print(data[])
    
    